In [1]:
from methods.svm import SVM
from preparers import ticker_svm
from core.functions import forecast
from core.simulation import simulate_trades_continuous

from loaders.alphavantage import get_ticker_data
from utils.viz import plot_time_series, plot_balance, plot_balance_vs_price

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.statespace.varmax import VARMAX

In [3]:
# Step 1: Data Loading
data_df = pd.read_csv('../dataset/insect/ant/location_in_mm.csv')
data_df = data_df[(data_df['colony_id']==1) & (data_df['ant_id']==12)]
data_df.reset_index(inplace = True)
data = data_df[['chamber', 'location_x', 'location_y']]
print(data)

      chamber  location_x  location_y
0           4       38.89      162.52
1           4       38.89      162.52
2           4       38.89      162.52
3           4       38.89      162.52
4           4       38.89      162.52
...       ...         ...         ...
2875        5        0.00        0.00
2876        5        0.00        0.00
2877        5        0.00        0.00
2878        5        0.00        0.00
2879        5        0.00        0.00

[2880 rows x 3 columns]


In [4]:
# Step 2: Data Preparation
#creating the train and validation set
train = data[:int(0.5*(len(data)))]
valid = data[int(0.5*(len(data))):]
print(train, valid)


      chamber  location_x  location_y
0           4       38.89      162.52
1           4       38.89      162.52
2           4       38.89      162.52
3           4       38.89      162.52
4           4       38.89      162.52
...       ...         ...         ...
1435        4        1.00      178.00
1436        4        1.00      178.00
1437        4        1.00      178.00
1438        4        1.00      178.00
1439        4        1.00      178.00

[1440 rows x 3 columns]       chamber  location_x  location_y
1440        4         1.0       178.0
1441        4         1.0       178.0
1442        4         1.0       178.0
1443        4         1.0       178.0
1444        4         1.0       178.0
...       ...         ...         ...
2875        5         0.0         0.0
2876        5         0.0         0.0
2877        5         0.0         0.0
2878        5         0.0         0.0
2879        5         0.0         0.0

[1440 rows x 3 columns]


In [ ]:
prediction_chamber = list()
prediction_x = list()
prediction_y = list()

start_t = len(train)
for t_i in tqdm(range(len(valid))):
    current_t = t_i + start_t
    model = VARMAX(data[t_i:current_t], order = (1,1))
    fitted_model = model.fit()
    prediction = fitted_model.forecast().reset_index(drop=True)
    prediction_chamber.append(prediction['chamber']) 
    prediction_x.append(prediction['location_x']) 
    prediction_y.append(prediction['location_y']) 
    


  0%|          | 0/1440 [00:00<?, ?it/s]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  0%|          | 1/1440 [00:06<2:35:48,  6.50s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: Converg

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  1%|          | 15/1440 [01:48<3:08:57,  7.96s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  1%|          | 16/1440 [01:55<3:03:56,  7.75s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation o

  2%|▏         | 29/1440 [03:32<2:48:12,  7.15s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  2%|▏         | 30/1440 [03:40<2:50:31,  7.26s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 44/1440 [05:14<2:23:23,  6.16s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 45/1440 [05:20<2:24:47,  6.23s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation o

  4%|▍         | 58/1440 [06:43<2:23:55,  6.25s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  4%|▍         | 59/1440 [06:50<2:24:52,  6.29s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:

...

  6%|▌         | 87/1440 [09:54<2:13:06,  5.90s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  6%|▌         | 88/1440 [09:59<2:08:48,  5.72s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  7%|▋         | 102/1440 [11:40<2:24:29,  6.48s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  7%|▋         | 103/1440 [11:47<2:23:04,  6.42s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation

  8%|▊         | 116/1440 [12:54<1:44:50,  4.75s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  8%|▊         | 117/1440 [12:59<1:46:16,  4.82s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.p

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  9%|▉         | 131/1440 [14:11<1:53:40,  5.21s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  9%|▉         | 132/1440 [14:16<1:56:54,  5.36s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation

 10%|█         | 145/1440 [15:22<1:41:10,  4.69s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 10%|█         | 146/1440 [15:28<1:47:13,  4.97s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.p

In [ ]:
#fit the model
model = VARMAX(train, order=(1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.forecast(steps = len(valid))
print(yhat)

In [ ]:
plot_time_series(ts_1=prediction_chamber, ts_label_1='VARMA Model', ts_2=valid['chamber'], ts_label_2='Close', title='VARMA predictions vs. ground truth')
plot_time_series(ts_1=prediction_x, ts_label_1='VARMA Model', ts_2=valid['location_x'], ts_label_2='Close', title='VARMA predictions vs. ground truth')
plot_time_series(ts_1=prediction_y, ts_label_1='VARMA Model', ts_2=valid['location_y'], ts_label_2='Close', title='VARMA predictions vs. ground truth')




In [37]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()
print(model_fit.summary())

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))
print(prediction)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 02, Jul, 2021
Time:                     17:46:10
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                  -0.329557
Nobs:                     1439.00    HQIC:                 -0.357107
Log likelihood:          -5844.81    FPE:                   0.688308
AIC:                    -0.373518    Det(Omega_mle):        0.682600
--------------------------------------------------------------------
Results for equation chamber
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const                 0.066815         0.012545            5.326           0.000
L1.chamber            0.843563         0.017878           47.186           0.000
L1.location_x         0.000723         0.000187           

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [32]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[4.97626288 0.04101043 0.09029971]]
